# Keras OCR 공개구현체의 Detector(CRAFT모델) Fine-tuning 실습

이 예제에서는 본인만의 데이터셋을 사용해서, OCR 공개구현체에서 문자 위치를 찾는 Detector 기능을 Fine-tuning하는 실습을 수행합니다.

여기 예제에서는 [여기](https://rrc.cvc.uab.es/?ch=1&com=downloads)에서 다운로드 받을 수 있는 ICDAR2013 문자인식용 데이터셋으로 Fine-tuning을 수행합니다.

Copyright: https://keras-ocr.readthedocs.io/en/latest/examples/fine_tuning_detector.html 의 공개구현체를 교육 목적에 맞게 수정 (2023.5.11, 김상호)

---

**(2024.11.18)**  tf.keras.callbacks.ModelCheckpoint() 함수에서 ValueError 발생하여 [링크](https://github.com/pythonlessons/mltu/issues/48) 의 방법에 따라 'save_weights_only=True'를 argument에 추가하고, checkpoint file의 이름 수정. model.fit() 함수의 arguments 중에서 workers도 blank 처리함.

**(2025.05.12)** imgaug 라이브러리와의 호환성 이슈때문에 numpy==1.25.0으로 다운그레이드



---



**(2025.05.12)** imgaug 라이브러리와의 호환성 이슈때문에 numpy==1.25.0으로 다운그레이드

In [ ]:
!pip install numpy==1.25.0


Keras OCR을 설치합니다.

In [ ]:
pip install keras_ocr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

OCR 라이브러리를 비롯해 다른 필요한 라이브러리들을 설치합니다.

'data_dir' 변수는 checkpoint file을 담고 있는 폴더로 지정합니다. (google drive의 폴더 위치)

In [3]:
data_dir = '/content/drive/MyDrive/object_detection'

import os
import math
import imgaug
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import tensorflow as tf

import keras_ocr

Keras에 내장된 ICDAR2013 문자인식용 데이터셋(이미지+문자 위치정보)을 읽어옵니다.

In [ ]:
dataset = keras_ocr.datasets.get_icdar_2013_detector_dataset(
    cache_dir='.',
    skip_illegible=False
)

dataset을 인쇄해서 어떤 형태로 구성되어 있는지 분석해봅니다.

공개구현체를 이용하기 위해서는, 본인의 데이터셋(이미지+문자위치 박스 정보)을 읽어와서 분석된 dataset과 같은 형태로 가공해야 합니다.

구체적으로는, dataset 형태를 분석해서, 위의 keras_ocr 라이브러리에서 내장 데이터셋을 읽어오는 get_icdar_2013_detector_dataset()함수와 유사하게 dataset을 return해주는 새로운 함수 get_mydata_detector_dataset() 함수를 만들어야 합니다.

In [ ]:
dataset

In [ ]:
dataset[0]

In [ ]:
dataset[0][1]

In [ ]:
dataset[0][1][0]

In [ ]:
dataset[0][1][0][0]

In [ ]:
dataset[0][1][0][0][0]

ICDAR2013 dataset이 아니라, 본인의 데이터셋을 읽어와서 detector를 학습하기 위해서는 여기 위치에서 2가지를 해야 합니다.

1. 본인만의 데이터셋을 업로드해서 unzip하고, training imge와 ground truth 파일을 ICDAR2013 dataset과 비교 (특히, ground truth 파일의 내용 비교 필요)
2. get_icdar_2013_detector_dataset()함수와 유사하게 dataset을 return해주는 새로운 함수 get_mydata_detector_dataset() 함수를 define (ground truth file에 들어가 있는 내용을 해석해서, 위에서 분석한 dataset 형식으로 가공하도록)



In [ ]:
def get_mydata_detector_dataset(cache_dir=None, skip_illegible=False):
  ...

In [ ]:
dataset = get_mydata_detector_dataset(cache_dir='.')

dataset이 의도했던 포맷으로 나오는지 확인합니다.

In [ ]:
dataset

읽어온 데이터셋은 Training:Validation = 80%:20%로 분할하고, 아래의 전처리를 수행합니다. 단, Validation Images는 augmentation은 적용하지 않습니다.

1.   Augmentation (random scaling/rotation/blurring/multiplication)
2.   이미지를 640*480 사이즈로 변경

In [6]:
train, validation = sklearn.model_selection.train_test_split(
    dataset, train_size=0.8, random_state=42
)
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.Affine(
      scale=(1.0, 1.2),
      rotate=(-5, 5)
    ),
    imgaug.augmenters.GaussianBlur(sigma=(0, 0.5)),
    imgaug.augmenters.Multiply((0.8, 1.2), per_channel=0.2)
])
generator_kwargs = {'width': 640, 'height': 480}
training_image_generator = keras_ocr.datasets.get_detector_image_generator(
    labels=train,
    augmenter=augmenter,
    **generator_kwargs
)
validation_image_generator = keras_ocr.datasets.get_detector_image_generator(
    labels=validation,
    **generator_kwargs
)

제대로 training용 dataset이 준비되었는지 Sanity check 목적으로, 입력 이미지중 하나를 문자위치 박스와 함께 화면에 보여줍니다.

In [ ]:
image, lines, confidence = next(training_image_generator)
canvas = keras_ocr.tools.drawBoxes(image=image, boxes=lines, boxes_format='lines')
plt.imshow(canvas)

Detector 모델을 Keras OCR 라이브러리를 사용해서 build하는데, 이때 Detector 모델의 초기 weights는 CRAFT모델의 pre-trained weights를 자동으로 읽어옵니다.

그 후에, 1000번의 epoch동안 fine-tuning 학습을 수행하면서, 매 학습차수(epoch)마다 Keras의 callback 라이브러리를 사용하여 아래 항목들을 수행합니다.
(만일, 10번으로 나눠서 학습을 하고싶다면, epochs 숫자를 100으로 줄여서 학습을 하고, 다시 학습을 시작할 때에 h5 weights file을 Colab에 업로드하고 아래 학습을 시작하는 방식으로 이어서 학습합니다)

1.   손실함수가 줄어들지 않으면 1000번 epoch을 모두 실행하기 이전에 조기 종료시키는 early stopping (학습이 충분하게 이루어지지 않을 경우, comment처리하거나 patience값을 키움)
2.   학습 과정을 지정된 csv 파일로 기록
3.  학습된 weights를 지정된 checkpoint파일에 기록



In [11]:
detector = keras_ocr.detection.Detector()
# if any, load the detector model weights
model_weights = os.path.join(data_dir,'detector_mydata.h5')
if os.path.isfile(model_weights) == True:
  detector.model.load_weights(model_weights)
  print(model_weights,': loaded successfully!')

Looking for /root/.keras-ocr/craft_mlt_25k.h5


**(2024.11.18)**  tf.keras.callbacks.ModelCheckpoint() 함수에서 ValueError 발생하여 [링크](https://github.com/pythonlessons/mltu/issues/48) 의 방법에 따라 'save_weights_only=True'를 argument에 추가하고, checkpoint file의 이름 수정. model.fit() 함수의 arguments 중에서 workers도 blank 처리함.

In [ ]:
# Run the re-training
batch_size = 1
training_generator, validation_generator = [
    detector.get_batch_generator(
        image_generator=image_generator, batch_size=batch_size
    ) for image_generator in
    [training_image_generator, validation_image_generator]
]
detector.model.fit(
    training_generator,
    steps_per_epoch=math.ceil(len(train) / batch_size),
    epochs=2,
#    workers=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5),
        tf.keras.callbacks.CSVLogger(os.path.join(data_dir, 'detector_icdar2013.csv')),
        tf.keras.callbacks.ModelCheckpoint(save_weights_only=True, filepath=os.path.join(data_dir, 'detector_icdar2013.weights.h5'))
    ],
    validation_data=validation_generator,
    validation_steps=math.ceil(len(validation) / batch_size)
)

In [16]:
!cp /content/drive/MyDrive/object_detection/detector_icdar2013.weights.h5 /content/drive/MyDrive/object_detection/detector_icdar2013.weights.h5.20250513

Detector의 fine-tuning 학습이 끝나면, 학습과정을 담고 있는 csv 로그 파일과, 학습결과 weights를 담고 있는 체크포인트 파일을 본인의 PC로 다운로드 받습니다. 아래 code에서 파일명은 위에서 지정된 checkpoint file의 위치로 수정해야 합니다.

In [ ]:
from google.colab import files
files.download("/content/detector_icdar2013.csv")
files.download("/content/detector_icdar2013.h5")

---

### 학습된 Weights로 Validation 데이터를 추론해보기

위의 100번 epoch 실행을 10번 반복 수행해서, 총 1000번 epoch만큼 학습이 완료된 'detector_mydata.h5' 파일을 Colab에 업로드해서 detection test를 수행해 봅시다.

아래 셀을 실행하면, 모든 validation 데이터셋에 대해 detection을 수행하고, 문자 위치 박스가 표시된 결과 이미지를 output_folder에 저장합니다.

학습을 몇번만 진행한 것과 충분히 학습한 Detector의 추론 성능을 아래와 같이 비교해보세요.

1. 먼저, 위에서 학습을 진행하지 않거나 몇번 epoch만 학습을 진행하고 중단한 후에, 아래의 셀을 실행해서 추론 성능을 test합니다.
2. 다음에, 1000번 학습이 완료된 weights 파일('detector.mydata.h5')을 업로드하고, 위로 돌아가서 detector를 다시 build하고, 아래의 셀을 실행해서 추론 성능을 test합니다.

In [12]:
!mkdir mydata_detection_result

In [ ]:
import imageio
import cv2

output_folder = 'mydata_detection_result'

for image_path, _, _ in validation:
  image = keras_ocr.tools.read(image_path)

  output_image_path = os.path.join(output_folder, image_path.split('/')[-1])

  # detector prediction
  pred_boxes = detector.detect(np.expand_dims(image, axis=0))

  for each_pred in pred_boxes[0]:
    left, top = each_pred[0]
    right, bottom = each_pred[2]
    canvas = cv2.rectangle(image, (int(left), int(top)), (int(right), int(bottom)), (0,255,0), 3)

  imageio.imwrite(output_image_path, canvas)
  print(output_image_path + ' saved!' )

Detection 추론 결과 폴더를 압축하고, 내 PC로 다운로드 받습니다.

In [ ]:
!zip -r /content/mydata_detection_result.zip /content/mydata_detection_result

from google.colab import files
files.download("/content/mydata_detection_result.zip")